In [1]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [2]:
from IPython.display import display, IFrame, HTML, Audio

from ipywidgets import widgets

import ipywidgets
import functools
import os
import random
from IPython.display import clear_output
from IPython.display import FileLink
from ipywidgets import Layout, Button, Box, VBox
class Character:
    def __init__(self, hp, dc, img_path, attack, dmg, img_path2= None):
        self.hp = hp
        self.dc = dc
        self.img = img_path
        self.img2 = img_path2
        self.attack = attack
        self.dmg = dmg

story_outline= {
    1:["next",  "battle1"],
    2:["next", "back"],
    3:["next", "back"],
    4:["next", "back"],
    5:["next", "back"],
    6:["next", "back"],
    7:["next", "back"],
    8:["next", "back"],
    9:["next", "back", "battle1"],
    10:["next", "back"],
    11:["next", "back"],
    12:["next", "back"],
    13:["next", "back", "battle2"],
    14:["next", "back"],
    15:["next", "back"],
    126:["next", "back"],
    17:["back"]
}
        
druid = Character(5, 5, os.path.join(f"resources", "Characters","druid.png"), 5, 1, os.path.join(f"resources", "Character_selection","druid.png"))

wizard = Character(3, 5, os.path.join(f"resources", "Characters","wizard.png"), 4, 2,  os.path.join(f"resources", "Character_selection","wizard.png")) 

rouge = Character(4, 8, os.path.join(f"resources", "Characters","rouge.png"), 3, 1, os.path.join(f"resources", "Character_selection","rouge.png"))

angel = Character(7, 6, os.path.join(f"resources", "Characters","angel.png"), 4, 1, os.path.join(f"resources", "Character_selection","angel.png"),) 

dragon = Character(4, 7, os.path.join(f"resources", "Characters","dragon.png"), 4, 1, os.path.join(f"resources", "Character_selection","dragon.png"))

warrior = Character(7, 7, os.path.join(f"resources", "Characters","warrior.png"), 3, 1, os.path.join(f"resources", "Character_selection","warrior.png")) 

monster1 = Character(9, 6, os.path.join(f"resources", "Characters","monster1.png"), 6, 1)
monster2 = Character(14, 4, os.path.join(f"resources", "Characters","monster2.png"), 4, 2)
monsters = {"monster1":monster1, "monster2":monster2}

characters = {"דרואיד":druid, "קוסם":wizard, "נוכל":rouge, "חצי מלאך":angel, "דם דרקון":dragon, "לוחם":warrior}
story= 1
intro = 0


def get_story():
    try:
        global story
        file = open(os.path.join(f"resources", "story", f"{story}.png"), "rb")
        story = story + 1
    except OSError:
        file = open(os.path.join(f"resources", "story", f"17.png"), "rb")
        
    image = file.read()
    return widgets.Image(
        value=image,
        #width=500,
        #height=600,
    )
def get_intro():
    global intro
    file = open(os.path.join(f"resources", "intro", f"{intro}.png"), "rb")
    intro = intro + 1
    image = file.read()
    return widgets.Image(
        value=image,
        #width=500,
        #height=600,
    )

def next_intro(b):
    global intro
    if intro == 2:
        choose_characters()
    else:
        run_intro()

def back_page(b):
    global story
    story = story-2
    run_story()


def next_page(b):
    can_pass_to_next_report = True#check_mandatory_fields(subj_results, task)
    if can_pass_to_next_report:
     
        run_story()
    else:
        b.button_style='warning'
        
def roll_dice(b, lbl=None):
    lbl.value = str(random.randrange(1,7))

def get_choises(characters):
    return [widgets.Label("בחרו דמויות:")]+[widgets.GridBox([widgets.Checkbox(description= i) for i in 
        characters.keys()])]

def choose_func(b, chars=None, targets_buttons=None):
    chosen = {i.description:i.value for i in targets_buttons[1].children}
    global characters
    characters = {i: characters[i] for i in characters.keys() if chosen[i]}
    run_story()

def show_characters(b, characters=characters, select=False):
    wids = []
    
    for char in characters.keys():
        if select:
            file = open(characters[char].img2, "rb")
        else: 
            file = open(characters[char].img, "rb")
        image = file.read()
        img = widgets.Image(
            value=image,
            width=500,
            height=600,
        )
        if select:
            wids.append(img)
        else: 
            lbl = widgets.Label(f"{char}, חיים = {characters[char].hp}")
            wids.append(widgets.GridBox([lbl, img]))

    display(widgets.GridBox([*wids]))


def get_targets(characters):
    return widgets.ToggleButtons(options=characters.keys())     

def hit_func(b, targets_buttons=None, characters=characters, dmg=1, play_turn_params=None):
    target = characters[targets_buttons.value]
    target.hp = target.hp - dmg
    play_turn(*play_turn_params)
    
def miss_func(b, play_turn_params=None):
    play_turn(*play_turn_params)
    
def attack(b, attacker=None, lbl=None, hit=None, miss=None, targets_buttons=None, characters=characters):
    target = characters[targets_buttons.value]
    if target is None:
        b.button_style='warning'
    else:
        dice =  random.randrange(1,7)
        atk = attacker.attack + dice
        
        lbl.value = f"התקפה = {atk}  הגנה = {target.dc}"
        hit.disabled = False
        miss.disabled = False
## screens    

def choose_characters():
    clear_output()
    targets_buttons = get_choises(characters)
    choose = widgets.Button(description=f"בחירה")
    choose.on_click(functools.partial(choose_func, chars=characters, targets_buttons=targets_buttons))
    chars = widgets.Button(description=f"הראה דמויות")
    chars.on_click(functools.partial(show_characters, characters=characters, select=True))
    
    display(widgets.GridBox([*targets_buttons, choose, chars]))
    
    
def run_story(): 
    clear_output()

    dice_lbl = widgets.Label("0")
    dice = widgets.Button(description=f"גלגול קוביה")
    dice.on_click(functools.partial(roll_dice, lbl=dice_lbl))
    
    chars = widgets.Button(description=f"הראה דמויות")
    chars.on_click(functools.partial(show_characters, characters=characters))
    global story
    
    widgts = []
    
    if "next" in story_outline[story]: 
        next_button = widgets.Button(description=f"הבא")
        next_button.on_click(functools.partial(next_page))
        widgts.append(next_button)

    if "back" in story_outline[story]:
        back_button = widgets.Button(description=f"הקודם")
        back_button.on_click(functools.partial(back_page))
        widgts.append(back_button)
        
    if "battle1" in story_outline[story]:
        battle_button = widgets.Button(description=f"קרב")
        battle_button.on_click(functools.partial(play_turn, battle_round = iter(["מפלצת", *characters.keys()]), idx=1))
        widgts.append(battle_button)
        
    if "battle2" in story_outline[story]:
        battle_button = widgets.Button(description=f"קרב")
        battle_button.on_click(functools.partial(play_turn, battle_round = iter(["מפלצת", *characters.keys()]), idx=2))
        widgts.append(battle_button)
        
    widgts.append(get_story())
    
    widgts.extend([chars, dice_lbl, dice])
    
    display(widgets.GridBox(widgts))
    
    
def play_turn(b, battle_round, idx):
    
    try:
        turn = next(battle_round)
        battle_characters = characters
        battle_characters["מפלצת"] = monsters[f"monster{idx}"]
        battle(turn, battle_characters, play_turn_params= [b, battle_round, idx])
    except StopIteration:
        battle_round = iter([*characters.keys()])
        play_turn(b, battle_round, idx)

        
def battle(turn, characters, play_turn_params=None):
    clear_output()
    turn_lbl = widgets.Label(f"התור של - {turn}")
        
    next_button = widgets.Button(description=f"סיום קרב")
    next_button.on_click(functools.partial(next_page))

    attacker = characters[turn]
    target = None
    turn_lbl1 = widgets.Label(f"האם ההתקפה פוגעת?")
    targets_buttons = get_targets(characters)
    hit = ipywidgets.Button(description="פגיעה", disabled=True)
    dmg = attacker.dmg
    hit.on_click(functools.partial(hit_func, targets_buttons=targets_buttons, characters=characters, dmg=dmg, play_turn_params=play_turn_params))

    miss = ipywidgets.Button(description="פספוס", disabled=True)
    miss.on_click(functools.partial(miss_func, play_turn_params=play_turn_params))
    
    attack_lbl = widgets.Label("")
    attack_button = widgets.Button(description=f"ביצוע התקפה")
    
    chars = widgets.Button(description=f"הראה דמויות")
    chars.on_click(functools.partial(show_characters, characters=characters))
    
    if turn == "מפלצת":
        turn_lbl2 = widgets.Label(f"את מי המפלצת תתקוף?")
        
        attack_button.on_click(functools.partial(attack, attacker=attacker, lbl=attack_lbl, hit=hit, miss=miss, targets_buttons=targets_buttons, characters=characters))
        display(widgets.GridBox([turn_lbl, turn_lbl2, targets_buttons, attack_lbl,attack_button, turn_lbl1, hit, miss, next_button, chars]))
        
        
    else: 
        
        targets_buttons.value = "מפלצת"
        attack_button.on_click(functools.partial(attack, attacker=attacker, lbl=attack_lbl, hit=hit, miss=miss, targets_buttons=targets_buttons, characters=characters))
        display(widgets.GridBox([turn_lbl, attack_lbl,attack_button, turn_lbl1, hit, miss, next_button, chars]))
        
def run_intro():
    clear_output()

    next_button = widgets.Button(description=f"הבא")
    next_button.on_click(functools.partial(next_intro))
    img = get_intro()
    display(widgets.GridBox([img, next_button]))
run_intro()    

GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xb0\x00\x00\x02\x12\x08\x02\x0…